# Notes
Regression model to directly predict returns

In [1]:
# import libraries
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
from os.path import isfile, isdir, join
import numpy as np
import pandas as pd
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
import re
from IPython.display import display
from zipfile import ZipFile
import pickle
import unicodedata
import pytz
from joblib import Parallel, delayed
import shutil
import difflib
import random
import math
from shutil import copyfile
import itertools
import time
from tqdm import tqdm
import collections
from collections import deque
import gc

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ndcg_score, accuracy_score, mean_squared_error

import lightgbm as lgbm
import optuna
from optuna import Trial, visualization

import matplotlib as mpl
from matplotlib import pyplot as plt

import jpx_tokyo_market_prediction

from utility_script import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [2]:
'''
Notebook Parameters
'''
SEED = 0
N_STOCK_SAMPLE = 2000
N_DAY_SAMPLE = 300
N_FOLD = 5
STOCK_LIST = pd.read_csv('../input/jpx-tokyo-stock-exchange-prediction/stock_list.csv').loc[lambda x: x.Universe0==1].SecuritiesCode.sort_values().unique().tolist()[:N_STOCK_SAMPLE]
LAGS = {'1d':1, '3d':3, '1w':5, '1m':20, '3m':20*3, '6m':20*6, '12m':20*12}
# LAGS = {'3d':3, '1w':5}
MAX_DAYS_LAG = max(list(LAGS.values()))
WIN_SIZE = 500
JPX_PATH = '../input/jpx-tokyo-stock-exchange-prediction'
TRN_SIZE = 240
VAL_SIZE = 60
TRN_VAL_GAP = 14
EVAL_METRIC = 'rmse'
DEVICE_TYPE = 'CPU'
EARLY_STOPPING_ROUNDS = 30


In [3]:
def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
    """
    Args:
        df (pd.DataFrame): predicted results
        portfolio_size (int): # of equities to buy/sell
        toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
    Returns:
        (float): sharpe ratio
    """
    def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
        """
        Args:
            df (pd.DataFrame): predicted results
            portfolio_size (int): # of equities to buy/sell
            toprank_weight_ratio (float): the relative weight of the most highly ranked stock compared to the least.
        Returns:
            (float): spread return
        """
        assert df['Rank'].min() == 0
        assert df['Rank'].max() == len(df['Rank']) - 1
        weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
        purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
        short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
        return purchase - short

    buf = df.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size, toprank_weight_ratio)
    sharpe_ratio = buf.mean() / buf.std()
    return sharpe_ratio

def rank_corr(df):
    return df.groupby('Date').apply(lambda x: x.iloc[:,1:].rank().corr().iloc[0,1]).mean()

def get_pred_df(header, raw_pred, y_true, reverse_order):
    df_pred = pd.concat([header[['Date','SecuritiesCode']].assign(Date=lambda x: x.Date.dt.strftime('%Y-%m-%d'), SecuritiesCode=lambda x: x.SecuritiesCode.astype(int)).reset_index(drop=True),
                        pd.Series(raw_pred).rename('raw_pred').reset_index(drop=True),
                        y_true.reset_index(drop=True)
                        ], axis=1)
    if reverse_order:
        df_pred['raw_pred'] = df_pred['raw_pred'] * -1
    df_pred['Rank'] = (df_pred.groupby('Date').raw_pred.rank(method='first') - 1).astype(int)
    return df_pred

In [4]:
# load features
features = load_pkl('../input/jpx-feature-engineering/features')
# reduce data size
features.iloc[:,3:] = features.iloc[:,3:].astype(np.float32)

In [5]:
'''
Train-Val split where validation periods are right after train period
'''
def trn_val_split_even_val(features):
    # define start date of each fold
    fold_dates = features.Date.drop_duplicates().reset_index(drop=True)
    fold_dates = fold_dates.shift(TRN_SIZE + TRN_VAL_GAP + VAL_SIZE).loc[lambda x: x.notnull()].reset_index(drop=True)
    fold_dates = fold_dates.loc[np.linspace(0, fold_dates.shape[0]-1, N_FOLD).astype(int)].reset_index(drop=True)

    # create and append train + val data
    full_dates = features.Date.drop_duplicates().reset_index(drop=True)
    full_data = []
    for i, date in enumerate(fold_dates):
        dates_trn = full_dates.loc[lambda x: x>=date].head(TRN_SIZE)
        dates_val = full_dates.loc[lambda x: x>=date].shift(-TRN_SIZE).shift(-TRN_VAL_GAP).head(VAL_SIZE)
        full_data.append(features.loc[lambda x: x.Date.isin(dates_trn)].assign(fold=i, trn_val='trn'))
        full_data.append(features.loc[lambda x: x.Date.isin(dates_val)].assign(fold=i, trn_val='val'))
    full_data = pd.concat(full_data).reset_index(drop=True)
    return full_data


'''
Train-Val split where validation periods are farthest in future
'''
def trn_val_split_late_val(features):
    # define start date of each fold
    fold_dates = features.Date.drop_duplicates().reset_index(drop=True)
    fold_dates = fold_dates.shift(TRN_SIZE + TRN_VAL_GAP + VAL_SIZE).loc[lambda x: x.notnull()].reset_index(drop=True)
    fold_dates = fold_dates.loc[np.linspace(0, fold_dates.shape[0]-1, N_FOLD).astype(int)].reset_index(drop=True)

    # create and append train + val data
    full_dates = features.Date.drop_duplicates().reset_index(drop=True)
    val_dates_list = full_dates.tail(VAL_SIZE * N_FOLD).values.reshape(N_FOLD, -1)
    full_data = []
    for i, date in enumerate(fold_dates):
        dates_trn = full_dates.loc[lambda x: x>=date].head(TRN_SIZE)
        dates_val = val_dates_list[i]
        full_data.append(features.loc[lambda x: x.Date.isin(dates_trn)].assign(fold=i, trn_val='trn'))
        full_data.append(features.loc[lambda x: x.Date.isin(dates_val)].assign(fold=i, trn_val='val'))
    full_data = pd.concat(full_data).reset_index(drop=True)
    return full_data

In [6]:
# train-val split
full_data = trn_val_split_late_val(features)

# add targets
targets = pd.concat([pd.read_csv(f'{JPX_PATH}/{folder}/stock_prices.csv', parse_dates=['Date']) for folder in ['train_files', 'supplemental_files']]) \
    .loc[:,['RowId','Target']] \
    .reset_index(drop=True)
full_data = full_data.merge(targets, how='inner', on='RowId')

# release memory
del features
gc.collect()

30

In [7]:
# define column types
id_cols = ['RowId','Date','SecuritiesCode','fold','trn_val']
all_features = [c for c in list(full_data) if c not in id_cols and c!='Target']
cat_features = ['zero_trade','supervision']
time_features = [c for c in all_features if '_mkt' in c] + \
                [c for c in all_features if c[:4]=='sec_'] + \
                [c for c in all_features if c[-4:] in ['_sin','_cos']]
stock_num_features = [c for c in all_features if c not in cat_features + time_features]

In [8]:
# scaling
for fold in range(N_FOLD):
    trn_idx = full_data.loc[lambda x: (x.fold==fold) & (x.trn_val=='trn')].index
    val_idx = full_data.loc[lambda x: (x.fold==fold) & (x.trn_val=='val')].index
    scaler = StandardScaler().fit(full_data.loc[trn_idx, time_features + stock_num_features])
    full_data.loc[trn_idx, time_features + stock_num_features] = scaler.transform(full_data.loc[trn_idx, time_features + stock_num_features])
    full_data.loc[val_idx, time_features + stock_num_features] = scaler.transform(full_data.loc[val_idx, time_features + stock_num_features])
    save_pkl(scaler, f'scaler{fold}')

In [9]:
'''
Function to train a model given the dataset definition, Hyperparameters, and mode
'''
def train_model(fold, params, hp_tune):
    # unpack model params
    selected_feats, lgbm_var_params, lgbm_fixed_params = params
    # define dataset
    data_trn = full_data.loc[lambda x: (x.fold==fold) & (x.trn_val=='trn')]
    data_val = full_data.loc[lambda x: (x.fold==fold) & (x.trn_val=='val')]
    X_trn, y_trn, header_trn = data_trn[selected_feats], data_trn.Target, data_trn[id_cols]
    X_val, y_val, header_val = data_val[selected_feats], data_val.Target, data_val[id_cols]
    # model fit
    model = lgbm.LGBMRegressor(**lgbm_fixed_params, **lgbm_var_params)
    model.fit(
        X = X_trn,
        y = y_trn,
        eval_set = [(X_val, y_val)],
        eval_names = ['Validation'],
        verbose = False,
        eval_metric = EVAL_METRIC,
        early_stopping_rounds = EARLY_STOPPING_ROUNDS
    )

    # output
    if hp_tune:
        model_output = model.best_score_['Validation'][EVAL_METRIC]
    else:
        # save model
        save_pkl(model, f"model{fold}")
        # prediction
        pred_trn = model.predict(X_trn)
        pred_val = model.predict(X_val)
        # prediction df
        df_pred_trn = get_pred_df(header_trn, pred_trn, y_trn, reverse_order=True)
        df_pred_val = get_pred_df(header_val, pred_val, y_val, reverse_order=True)
        # sharpe
        sharpe_trn = calc_spread_return_sharpe(df_pred_trn)
        sharpe_val = calc_spread_return_sharpe(df_pred_val)
        # output dict
        model_output = dict(
            fold = fold,
            params = params,
            best_iteration_ = model.best_iteration_,
            best_score_ = model.best_score_['Validation'][EVAL_METRIC],
            sharpe = {'trn': sharpe_trn, 'val': sharpe_val},
            ann_sharpe = {'trn': sharpe_trn * np.sqrt(240), 'val': sharpe_val * np.sqrt(240)},
            evals_result_ = model.evals_result_['Validation'][EVAL_METRIC],
            feat_imp = pd.DataFrame(list(zip(model.feature_name_, model.feature_importances_)), columns=['feat','imp']),
        )
    return model_output 

In [10]:
# selected_feats = [c for c in all_features if c[:5]=='price']
# lgbm_var_params = dict(
#     max_depth = 8,
#     num_leaves = 32,
#     learning_rate = 0.01,
#     min_child_weight = 1,
#     min_child_samples = 50,
#     subsample = 0.8,
#     subsample_freq = 4,
#     colsample_bytree = 0.8,
#     reg_alpha = 2,
#     reg_lambda = 30,
# )

# lgbm_fixed_params = dict(
#     objective = 'rmse',
#     min_split_gain = 0,
#     n_estimators = 5000,
#     boosting_type = 'gbdt',
#     n_jobs = -1,
#     random_state = SEED,
#     device_type = DEVICE_TYPE
# )

# # pack all params
# params = (selected_feats, lgbm_var_params, lgbm_fixed_params)
# train_model(fold=0, params=params, hp_tune=False)

In [11]:
# full_data.loc[lambda x: (x.fold==0) & (x.trn_val=='val')].Date.describe()
# model = load_pkl('model0')
# data_val = full_data.loc[lambda x: x.Date>='2020-02-04']
# X_val, y_val, header_val = data_val[selected_feats], data_val.Target, data_val[id_cols]
# pred_val = model.predict(X_val)
# df_pred_val = get_pred_df(header_val, pred_val, y_val, reverse_order=True)
# sharpe_val = calc_spread_return_sharpe(df_pred_val)

# def _calc_spread_return_per_day(df, portfolio_size, toprank_weight_ratio):
#     assert df['Rank'].min() == 0
#     assert df['Rank'].max() == len(df['Rank']) - 1
#     weights = np.linspace(start=toprank_weight_ratio, stop=1, num=portfolio_size)
#     purchase = (df.sort_values(by='Rank')['Target'][:portfolio_size] * weights).sum() / weights.mean()
#     short = (df.sort_values(by='Rank', ascending=False)['Target'][:portfolio_size] * weights).sum() / weights.mean()
#     return purchase - short

# # def calc_spread_return_sharpe(df: pd.DataFrame, portfolio_size: int = 200, toprank_weight_ratio: float = 2) -> float:
# buf = df_pred_val.groupby('Date').apply(_calc_spread_return_per_day, portfolio_size=200, toprank_weight_ratio=2)

In [12]:
# buf.rolling(60).apply(lambda s: s.mean()/s.std()).plot()
# buf.cumsum().plot()

In [13]:
# for lr in [0.01]:
#     selected_feats = [c for c in all_features if c[:5]=='price']
#     lgbm_var_params = dict(
#         max_depth = 8,
#         num_leaves = 32,
#         learning_rate = lr,
#         min_child_weight = 1,
#         min_child_samples = 50,
#         subsample = 0.8,
#         subsample_freq = 4,
#         colsample_bytree = 0.8,
#         reg_alpha = 2,
#         reg_lambda = 30,
#     )

#     lgbm_fixed_params = dict(
#         objective = 'rmse',
#         min_split_gain = 0,
#         n_estimators = 5000,
#         boosting_type = 'gbdt',
#         n_jobs = -1,
#         random_state = SEED,
#         device_type = DEVICE_TYPE
#     )

#     # pack all params
#     params = (selected_feats, lgbm_var_params, lgbm_fixed_params)

#     # loop through all folds
#     model_outputs = []
#     for fold in range(2,3):
#         model_outputs.append(train_model(fold, params, hp_tune=False))
#     sharpe_trn = np.mean([x['sharpe']['trn'] for x in model_outputs]) * 240**0.5
#     sharpe_val = np.mean([x['sharpe']['val'] for x in model_outputs]) * 240**0.5
#     best_iteration_ = [x['best_iteration_'] for x in model_outputs]
#     print(lr)
#     print(sharpe_trn, sharpe_val)
#     print(best_iteration_)

In [14]:
# !pip install BorutaShap
# from BorutaShap import BorutaShap
# from sklearn.ensemble import RandomForestRegressor
# model = RandomForestRegressor(n_estimators=3)
# fold = 0
# data_trn = full_data.loc[lambda x: (x.fold==fold) & (x.trn_val=='trn')].sample(10000)
# # data_val = full_data.loc[lambda x: (x.fold==fold) & (x.trn_val=='val')]
# X_trn, y_trn, header_trn = data_trn[all_features[:]], data_trn.Target, data_trn[id_cols]
# # X_val, y_val, header_val = data_val[selected_feats], data_val.Target, data_val[id_cols]
# Feature_Selector = BorutaShap(importance_measure='shap', classification=False, model=model)
# Feature_Selector.fit(X=X_trn, y=y_trn, n_trials=10, sample=False, train_or_test='test', normalize=True)

# Forward Selection

In [15]:
def get_feat_groups(all_feats, groups):
    all_feats = pd.Series(all_feats)
    feat_groups = []
    for grp in groups:
        lst = []
        for p in grp:
            feats = all_feats.loc[lambda x: x.str.startswith(p)]
            lst.append(feats)
            all_feats = all_feats.loc[lambda x: ~x.isin(feats)]
        feat_groups.append(pd.concat(lst).tolist())
    return feat_groups

groups = [
    ['zero_trade','beta','supervision','div_ratio','RS_sqrt_vol','days_since_last_fin'],
    ['price_'],
    ['volume_'],
    ['spread_'],
    ['volatility_'],
    ['ret_mkt','vol_mkt'],
    ['exret_'],
    ['dollar_'],
    ['sharpe_'],
    ['sec_cross_sect_'],
    ['sec_volume_'],
    ['sec_volatility_'],
    ['sec_dollar_'],
    ['day_in_','week_in'],
    ['NetSales_'],
    ['OperatingProfit_'],
    ['OrdinaryProfit_'],
    ['Profit_'],
    ['EarningsPerShare_'],
    ['TotalAssets_'],
    ['Equity_'],
    ['EquityToAssetRatio_'],
    ['BookValuePerShare_']
]

feat_groups = get_feat_groups(all_features, groups)
log(f'Total number of features considered: {sum([len(x) for x in feat_groups])}')
log(f'Total numnber of feature groups: {len(feat_groups)}')

[2022-05-29 23:31:21] Total number of features considered: 96
[2022-05-29 23:31:21] Total numnber of feature groups: 23


In [16]:
fs_lgbm_var_params = dict(
    max_depth = 8,
    num_leaves = 32,
    learning_rate = 0.01,
    min_child_weight = 1,
    min_child_samples = 50,
    subsample = 0.8,
    subsample_freq = 4,
    colsample_bytree = 0.8,
    reg_alpha = 1,
    reg_lambda = 6,
)

fs_lgbm_fixed_params = dict(
    objective = 'rmse',
    min_split_gain = 0,
    n_estimators = 5000,
    boosting_type = 'gbdt',
    n_jobs = -1,
    random_state = SEED,
    device_type = DEVICE_TYPE
)

In [17]:
def forward_selection(feat_groups):
    '''
    features: list of feature groups (a list of feature subsets)
    model_train_func: 
    '''
    base_feats = []
    hist = []
    while len(feat_groups) > 0:
        scores = []
        for feats in feat_groups:
            model_outputs = [train_model(fold=fold, params=(base_feats + feats, fs_lgbm_var_params, fs_lgbm_fixed_params), hp_tune=False) for fold in range(N_FOLD)]
            mean_score = np.mean([x['ann_sharpe']['val'] for x in model_outputs])
            scores.append(mean_score)
            hist.append((mean_score,
                         len(feat_groups),
                         len(base_feats),
                         ', '.join(base_feats), 
                         ', '.join(feats), 
                         ', '.join(base_feats + feats)
                         ))
            log(f'Score: {mean_score}; Remaining feature groups: {len(feat_groups)}; Features: {", ".join(base_feats + feats)}')
        best_score = max(scores)
        best_group = feat_groups[np.argmax(scores)]
        base_feats += best_group
        del feat_groups[np.argmax(scores)]
    hist = pd.DataFrame(hist, columns=['score','len_feat_groups','len_base_feats','base_feats','feats','total_feats'])
    return hist

fs_hist = forward_selection(feat_groups)
fs_hist.to_csv('fs_hist.csv', index=False)

[2022-05-29 23:32:08] Score: 0.27222914482628474; Remaining feature groups: 23; Features: zero_trade, beta, supervision, div_ratio, RS_sqrt_vol, days_since_last_fin
[2022-05-29 23:32:53] Score: -0.6006270224810267; Remaining feature groups: 23; Features: price_ma_pctg_ch_3d, price_ma_pctg_ch_1w, price_standard_dist_1m, price_standard_dist_3m, price_standard_dist_6m, price_standard_dist_12m
[2022-05-29 23:33:54] Score: 0.23895766496599008; Remaining feature groups: 23; Features: volume_ma_pctg_ch_3d, volume_ma_pctg_ch_1w, volume_standard_dist_1m, volume_standard_dist_3m, volume_standard_dist_6m, volume_standard_dist_12m
[2022-05-29 23:34:42] Score: -0.15407492377622622; Remaining feature groups: 23; Features: spread_ma_pctg_ch_3d, spread_ma_pctg_ch_1w, spread_standard_dist_1m
[2022-05-29 23:35:25] Score: -0.4737223932390048; Remaining feature groups: 23; Features: volatility_1w, volatility_1m, volatility_3m, volatility_12m, volatility_diff
[2022-05-29 23:36:05] Score: -0.316586357188005